In [1]:
# On utilise une autre méthode en réalisant directement un point de montage avec "gdrive" = google drive
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
import os
os.chdir("gdrive/My Drive/carrefour_word2vec_transfer_learning")
!cd data;ls

2019-12-28T11-12-50-593898100Z.sav
all_results_final_all_with_filter.pkl
cc.fr.300.vec
cc.fr.300.vec_of_titles.txt
fichiers_générés_par_Hervé
model.txt
oov_titles_bio.txt
save.dict_raw_entity_entity_cleansed
sentences_BIO.csv
text8
text8.zip
words_in_titles_BIO_and_not_in_corpus.txt


In [0]:
import pandas as pd
pd.set_option('display.max_colwidth',-1)
import csv
import numpy as np
import random
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.cluster import DBSCAN
from functions import fun_step_1_create_df as step_1

import sklearn
import re

**Le but de ce notebook est de calculer les embeddings de tous les mots présents dans les titres BIO**

**On peut utiliser deux méthodes:**
- méthode n°1: utiliser FastText de manière à calculer les embeddings OOV (Out Of Vocabulary)
- méthode n°2: utiliser Word2Vec + Skipgram et faire du Transfer learning grâce à Tensorflow / Keras

### Dans ce notebook, on utilise la méthode n°1 : obtention des embeddings pré-entrainés avec "FastText" à partir du corpus 

Fonction "read_fasttext_vecs":
- comme input le fichier .txt qui contient les embeddings ie tous les mots et les vecteurs des mots du corpus
- renvoit comme output les dictionnaires:
    - clés = mots, valeurs = index des mots
    - clés = index des mots, valeurs = mots
    - clés = mots, valeurs = vecteurs des mots

In [0]:
# cf "read_glove_vecs" function MOOC Emoji = exactement le meme code 
def read_fasttext_vecs(vectors_file):
    with open(vectors_file, 'r') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            #print (np.array(line[1:], dtype=np.float64))
            #print (line[1:])
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map

In [5]:
%%time
word_to_index, index_to_word, word_to_vec_map=read_fasttext_vecs("data/cc.fr.300.vec")

CPU times: user 2min 42s, sys: 7.67 s, total: 2min 49s
Wall time: 3min 3s


In [6]:
print ("On connaît maintenant" , len(word_to_vec_map), "embeddings pour tous les mots du gros corpus pré-entraîné")

On connaît maintenant 2000000 embeddings pour tous les mots du gros corpus pré-entraîné


On extrait tous les mots distincts présents dans tous les titres "bruts" du BIO<br>
On doit nettoyer la colonne "xtitle" en remplaçant "\xa0" par " "

In [10]:
# orgc_crf_category = step_1.create_df("orgc_crf_category", "data/fichiers_générés_par_Hervé/crf/","utf8")
orgc_crf_product = step_1.create_df("orgc_crf_product", "data/fichiers_générés_par_Hervé/crf/","utf8")
orgc_crf_product['breadcrumb_last_level'] = orgc_crf_product['xbreadcrumb'].map(lambda x: x.split(' > ')[-1])
# breadcrumbs_crf_last_level = list(set(list(orgc_crf_product['breadcrumb_last_level'])))
# len(breadcrumbs_crf_last_level)

# orgc_gwz_category = step_1.create_df("orgc_gwz_category",root_path + "/carrefour_chunking_bio/fichiers_générés_par_Hervé/gwz/","utf8")
orgc_gwz_product = step_1.create_df("orgc_gwz_product", "data/fichiers_générés_par_Hervé/gwz/","utf8")
orgc_gwz_product['breadcrumb_last_level'] = orgc_gwz_product['xbreadcrumb'].map(lambda x: x.split(' > ')[-1])
# breadcrumbs_gwz_last_level = list(set(list(orgc_gwz_product['breadcrumb_last_level'])))
# len(breadcrumbs_gwz_last_level)

# orgc_ntr_category = step_1.create_df("orgc_ntr_category",root_path + "/carrefour_chunking_bio/fichiers_générés_par_Hervé/ntr/","utf8")
orgc_ntr_product = step_1.create_df("orgc_ntr_product", "data/fichiers_générés_par_Hervé/ntr/","utf8")
orgc_ntr_product['breadcrumb_last_level'] = orgc_ntr_product['xbreadcrumb'].map(lambda x: x.split(' > ')[-1])
# breadcrumbs_ntr_last_level = list(set(list(orgc_gwz_product['breadcrumb_last_level'])))
# len(breadcrumbs_ntr_last_level)

# orgc_wbe_category = step_1.create_df("orgc_wbe_category",root_path + "/carrefour_chunking_bio/fichiers_générés_par_Hervé/wbe/","utf8")
orgc_wbe_product = step_1.create_df("orgc_wbe_product", "data/fichiers_générés_par_Hervé/wbe/","utf8")
orgc_wbe_product['breadcrumb_last_level'] = orgc_wbe_product['xbreadcrumb'].map(lambda x: x.split(' > ')[-1])
# breadcrumbs_wbe_last_level = list(set(list(orgc_wbe_product['breadcrumb_last_level'])))
# len(breadcrumbs_wbe_last_level)

frames = [orgc_crf_product, orgc_gwz_product, orgc_ntr_product, orgc_wbe_product]
orgc_descriptions = pd.concat(frames)

del frames
del orgc_crf_product, orgc_gwz_product, orgc_ntr_product, orgc_wbe_product 

orgc_descriptions.sample(n=1)

,xid,xtitle,xbrand,xbreadcrumb,xsubtitle,xdescription,xslug,ximg,xmeta_title,xmeta_description,xmeta_keywords,xapi,xweb,xcrawled_at,xurl,xrun,breadcrumb_last_level
7137,my-e-boisson-energisante-my-e-bio-250ml-p23258,Boisson énergisante MY E. BIO 250ml,My E.,Accueil > Santé & Bien-être > Sport et Nutrition > Energie et endurance > Boissons énergétiques sportifs,"MY E., la boisson énergisante BIO, 100% naturelle et sans aucun additif chimique ! Sans taurine. Canette de 250ml. MY E., la boisson énergisante BIO, 100% naturelle et sans aucun additif chimique ! Sans taurine. Canette de 250ml.","MY E., la boisson énergisante BIO, 100% naturelle et sans aucun additif chimique ! Sans taurine. Canette de 250ml.MY E., la boisson énergisante BIO, 100% naturelle et sans aucun additif chimique ! Sans taurine. Canette de 250ml.",my-e-boisson-energisante-my-e-bio-250ml-p23258,https://static.greenweez.com/images/products/23000/600/my-e-boisson-energisante-my-e-bio-250ml.jpg,,"Boisson énergisante MY E. BIO 250ml - Boissons énergétiques sportifs - Energie et endurance - MY E., la boisson énergisante BIO, 100% naturelle et sans aucun additif chimique ! Sans taurine. Canette de 250ml.",,v1.0,gwz,2019-01-17 19:43:00,https://www.greenweez.com/my-e-boisson-energisante-my-e-bio-250ml-p23258,19-01-17,Boissons énergétiques sportifs


In [0]:
orgc_descriptions['xtitle_without_xa0'] = orgc_descriptions['xtitle'].map(lambda x:re.sub('\xa0',' ',x))

In [12]:
%%time

words_titles_BIO = []
for row in orgc_descriptions.itertuples():
#     title_temp = row.xtitle.split(' ')
    title_temp = row.xtitle_without_xa0.split(' ')
    for each in title_temp:
        if each not in words_titles_BIO:
#         if each not in words_titles_BIO and '\xa0' not in each:
            # if each == 'Assiette Légumes':
                # print (title_temp)
            words_titles_BIO.append(each)

CPU times: user 2.65 s, sys: 3.89 ms, total: 2.66 s
Wall time: 2.67 s


In [13]:
print ("Il y a" , len(words_titles_BIO), "mots distincts dans les titres BIO")

Il y a 8761 mots distincts dans les titres BIO


Extraction de tous les mots à la fois dans le corpus et dans tous les titres du BIO

In [15]:
%%time

words_in_titles_BIO_and_corpus = []
for idx in list(range(1,2000001)):
    if index_to_word[idx] in words_titles_BIO:
        words_in_titles_BIO_and_corpus.append(index_to_word[idx])

CPU times: user 3min 41s, sys: 51.4 ms, total: 3min 41s
Wall time: 3min 41s


In [16]:
print ("Il y a", len(words_in_titles_BIO_and_corpus), "mots présents à la fois dans le gros corpus et dans les titres BIO")

Il y a 6444 mots présents à la fois dans le gros corpus et dans les titres BIO


Extraction de tous les mots présents dans tous les titres du BIO et absents du corpus:<br>
On fait "words_titles_BIO "moins "words_in_titles_BIO_and_corpus"

In [0]:
# On doit trouver : 
# 8761 - 6444 = 2317

In [17]:
%%time

words_in_titles_BIO_and_not_in_corpus = []

for each in words_titles_BIO:
    if each not in words_in_titles_BIO_and_corpus:
        words_in_titles_BIO_and_not_in_corpus.append(each) 

CPU times: user 754 ms, sys: 981 µs, total: 755 ms
Wall time: 761 ms


In [19]:
print ("Il y a", len(words_in_titles_BIO_and_not_in_corpus), "mots présents dans les titres BIO et absents du gros corpus")

Il y a 2317 mots présents dans les titres BIO et absents du gros corpus


On regarde si les mots ci-dessous sont présents ou absents du gros corpus

In [21]:
for word in ['sable', 'sables', 'Sable', 'Sables', 'découverte', 'Découverte','Découvertes','crème', 'Crème', 'Creme']:
  try:
    print (word_to_index[word])
  except:
    print (word, "absent du gros corpus")

1843110
1843141
1103512
1103523
1494144
545028
545039
1458217
490791
487719


NB: grâce au calcul des embeddings des mots de type OOV, on a obtenu des embeddings pour "Dosett'" ou encore '"Breakfast'<br>
Cf le notebook visualize_words_of_titles_with_fasttext.ipynb, ces mots "Dosett'" et '"Breakfast' font partie de clusters dont les autres mots sont seulement présents dans le gros corpus<br>
Par conséquent, le calcul des embeddings pour les mots OOV est efficace car il permet d'affecter ces mots à des classes du clustering.

In [0]:
# [('dosettes', 989), ('Dosettes', 989), ("Dosett'", 989)],

In [0]:
"Dosett'" in words_in_titles_BIO_and_corpus # renvoit False
"Dosett'" in words_in_titles_BIO_and_not_in_corpus

True

In [0]:
"Dosett'" in words_in_titles_BIO_and_corpus

In [0]:
'"Breakfast' obtenu dans la classe  [('Breakfast', 651), ('"Breakfast', 651)],

In [0]:
'"Breakfast' in words_in_titles_BIO_and_corpus

False

In [0]:
'"Breakfast' in words_in_titles_BIO_and_not_in_corpus

True

In [0]:
'dosettes' in words_in_titles_BIO_and_corpus
# 'Dosettes'

True

In [0]:
'dosettes' in words_in_titles_BIO_and_not_in_corpus

False

In [22]:
random.sample(words_in_titles_BIO_and_not_in_corpus,20)

['Toasté',
 'CRUSCANA',
 'Milkervé',
 'GRANAROLO',
 'orientau',
 'Chocolinette',
 'Pasteli',
 'Pomme-Banane',
 '(riz)',
 'corps/cheveux',
 '-180g',
 'cristal"',
 'jour/nuit',
 'Chocobanane',
 'vinettes',
 'maxipack',
 'TRAMIER',
 'Nutriamande',
 'Tofu,',
 'Biscottate']

**On crée le fichier texte de "words_in_titles_BIO_and_not_in_corpus"**

In [0]:
%%time
file = open("data/words_in_titles_BIO_and_not_in_corpus.txt","w")
for each in words_in_titles_BIO_and_not_in_corpus:
    file.write(each) 
    file.write("\n")
file.close() 

CPU times: user 2.29 ms, sys: 952 µs, total: 3.24 ms
Wall time: 33.5 ms


**CONSTAT SUR CE QU'ON PEUT FAIRE** :<br>
    
    - ignorer les mots inconnus, mais on va perdre de l'information (On considèrerait des descriptions sans "100%")

    - cf la doc FastText + OOV = Out Of Vocabulary
      On peut calculer des embeddings pour ces mots OOV inconnus grâce à FastText
    
    - ou faire du Transfer Learning grâce à des embeddings pré-entrainés
      du coup, logiquement si on veut entrainer les embeddings des mots "inconnus", il faut partir d'embeddings pré-entrainés grâce à une méthode connue comme word2vec / skipgrams ou Glove
    
    - soit considérer tous les mots "inconnus" en tirant aléatoirement leurs embeddings ou en initialisant tous leurs embeddings à zéro
      Puis on peut entrainer directement le modèle "downstream", en utilisant une option du type "trainable = yes"
        


**On choisit de faire le calcul des embeddings grâce à "FastText" de ces mots appelés "OOV"**<br>
Ce sont les mots dits Out Of Vocabulary si on considère que le "Vocabulary" = ensemble des mots du corpus de FastText

Installation de FastText en ligne de commandes avec:<br>
    - conda activate <env_name><br>
    - conda install -c conda-forge fasttext cf https://anaconda.org/conda-forge/fasttext<br>
    - aller dans /Documents/vincent/jupyter/carrefour_bio/notebooks/data<br>
    - lancer : <br>
    fasttext print-word-vectors cc.fr.300.bin < words_in_titles_BIO_and_not_in_corpus.txt > oov_titles_bio.txt<br>
        
Remarque: cette dernière commande fasttext a fonctionné dans l'environnement conda GPU<br>
mais n'a pas fonctionné dans l'environnement data: j'utilise le fichier que j'avais déjà généré

**Maintenant, on crée un dictionnaire contenant les embeddings des mots à la fois présents dans les titres et dans le gros corpus**

In [0]:
%%time
words_embeddings_in_titles_BIO_and_corpus = {}

with open("data/cc.fr.300.vec", "r") as f:
    # Remarque: la 1ère ligne est: 2000000 300
    #i = 0
    for line in f:
        #print (line)
        #i += 1
        line = line.strip().split(' ')
        # l[0] représente chaque mot
        curr_word = line[0]
        if curr_word in words_in_titles_BIO_and_corpus:
            words_embeddings_in_titles_BIO_and_corpus[curr_word] = np.array(line[1:], dtype=np.float64)
        #if i == 3:
            #break

CPU times: user 6min 16s, sys: 4.7 s, total: 6min 21s
Wall time: 6min 22s


In [0]:
len(words_embeddings_in_titles_BIO_and_corpus)

6444

In [0]:
random.sample(words_embeddings_in_titles_BIO_and_corpus.items(), 1 )

[('Ayurvédiques',
  array([ 0.0525, -0.0281,  0.0354, -0.0372, -0.0086, -0.0072, -0.0154,
          0.0285, -0.0186,  0.0433, -0.0119, -0.0648, -0.0032, -0.0294,
          0.0618,  0.0184, -0.0064,  0.0125, -0.0101,  0.0026, -0.0624,
         -0.0045, -0.0252,  0.0174, -0.014 ,  0.0246, -0.0511, -0.004 ,
          0.0154,  0.0565, -0.0255, -0.0288, -0.0342, -0.0152, -0.04  ,
          0.0083,  0.0157,  0.0167, -0.0144, -0.0473, -0.0157, -0.0016,
         -0.0394,  0.0301, -0.0164,  0.0113,  0.0445,  0.0223,  0.0285,
          0.0556, -0.0146, -0.0187,  0.0076,  0.0184,  0.0281, -0.0486,
          0.0006, -0.0292, -0.0079, -0.0009, -0.0313,  0.0131, -0.0152,
         -0.0048, -0.0344, -0.0122,  0.0324,  0.0152, -0.0144, -0.0185,
         -0.0051,  0.035 ,  0.0452,  0.0209, -0.0263,  0.0387,  0.0182,
          0.0672, -0.0803,  0.0087, -0.0102,  0.0058,  0.0171, -0.0281,
         -0.0181, -0.0137, -0.003 , -0.006 ,  0.0461, -0.013 ,  0.0306,
         -0.044 ,  0.0069,  0.0014,  0.0253,  

Création du dictionnaire "word_to_vec_map_words_in_titles_bio" contenant les embeddings :
- des mots à la fois présents dans les titres BIO et dans le gros corpus
- des mots présents dans les titres BIO mais absents du gros corpus (mots OOV)

In [0]:
%%time
dict_oov_titles_bio = {}
with open("data/oov_titles_bio.txt", "r") as f:
    for line in f:
        line = line.strip().split(' ')
        curr_word = line[0]
        dict_oov_titles_bio[curr_word] = np.array(line[1:], dtype=np.float64)

CPU times: user 182 ms, sys: 6.95 ms, total: 189 ms
Wall time: 660 ms


In [0]:
len(dict_oov_titles_bio)

2316

Création du dictionnaire contenant les embeddings de tous les mots des titres BIO

In [0]:
%%time
word_to_vec_map_words_in_titles_bio = {}
for k,v in words_embeddings_in_titles_BIO_and_corpus.items():
    word_to_vec_map_words_in_titles_bio[k] = v
for k,v in dict_oov_titles_bio.items():
    word_to_vec_map_words_in_titles_bio[k] = v

CPU times: user 3.63 ms, sys: 0 ns, total: 3.63 ms
Wall time: 3.63 ms


In [0]:
len(word_to_vec_map_words_in_titles_bio)

8760

### Sauvegarde du dictionnaire word_to_vec_map_words_in_titles_bio vers le fichier "data/cc.fr.300.vec_of_titles"

In [0]:
# On affiche les 2 premières lignes du fichier "cc.fr.300.vec" qui contient les embeddings du gros corpus
!cd data;ls;head -2 cc.fr.300.vec

2019-12-28T11-12-50-593898100Z.sav  save.dict_raw_entity_entity_cleansed
cc.fr.300.vec			    sentences_BIO.csv
cc.fr.300.vec_of_titles.txt	    text8
fichiers_générés_par_Hervé	    text8.zip
model.txt			    words_in_titles_BIO_and_not_in_corpus.txt
oov_titles_bio.txt
2000000 300
, 0.0058 0.0478 0.1094 -0.0839 -0.2092 0.0072 -0.0780 0.0683 0.0120 -0.0314 -0.0695 -0.0938 -0.0006 0.0257 0.0215 0.1130 0.0517 0.0191 -0.0224 -0.0168 0.0723 0.0711 -0.0505 -0.0987 -0.0960 -0.0695 0.0191 -0.0003 -0.1440 -0.0528 0.0305 0.0586 -0.0246 0.0195 -0.0040 0.0421 -0.0361 0.0546 0.1568 0.0482 -0.0072 -0.0352 -0.0004 0.1192 0.1274 0.1168 -0.0188 -0.0482 0.0467 0.0487 -0.0213 -0.0177 -0.0399 0.0466 0.0376 -0.0011 0.0841 0.0149 -0.2848 0.0367 0.0917 0.0908 0.0493 -0.1145 0.0352 -0.0179 -0.0245 0.0516 0.0297 0.0141 -0.0582 -0.0562 -0.1111 -0.0624 -0.1561 -0.0105 0.0271 -0.0011 0.0857 0.0516 -0.0387 -0.0856 0.0198 -1.1291 -0.0349 -0.0315 0.0705 -0.0057 -0.0195 -0.0522 0.0336 -0.0265 0.0823 0.0362 0.0892 -0

In [0]:
%%time

# On crée ce fichier sans entête contenant le nombre de mots + dimension des embeddings
file = open("data/cc.fr.300.vec_of_titles.txt","w")
for k,v in word_to_vec_map_words_in_titles_bio.items():
    # if k == "Croq'tomate" or  k == "Croq'tomate":
      # print (k)
    file.write(k + " ") 
    for each in v:
      file.write(str(each) + " ")
    file.write("\n")
file.close() 

CPU times: user 2.31 s, sys: 15 ms, total: 2.32 s
Wall time: 2.41 s


In [0]:
!cd data;head -5 cc.fr.300.vec_of_titles.txt

, 0.0058 0.0478 0.1094 -0.0839 -0.2092 0.0072 -0.078 0.0683 0.012 -0.0314 -0.0695 -0.0938 -0.0006 0.0257 0.0215 0.113 0.0517 0.0191 -0.0224 -0.0168 0.0723 0.0711 -0.0505 -0.0987 -0.096 -0.0695 0.0191 -0.0003 -0.144 -0.0528 0.0305 0.0586 -0.0246 0.0195 -0.004 0.0421 -0.0361 0.0546 0.1568 0.0482 -0.0072 -0.0352 -0.0004 0.1192 0.1274 0.1168 -0.0188 -0.0482 0.0467 0.0487 -0.0213 -0.0177 -0.0399 0.0466 0.0376 -0.0011 0.0841 0.0149 -0.2848 0.0367 0.0917 0.0908 0.0493 -0.1145 0.0352 -0.0179 -0.0245 0.0516 0.0297 0.0141 -0.0582 -0.0562 -0.1111 -0.0624 -0.1561 -0.0105 0.0271 -0.0011 0.0857 0.0516 -0.0387 -0.0856 0.0198 -1.1291 -0.0349 -0.0315 0.0705 -0.0057 -0.0195 -0.0522 0.0336 -0.0265 0.0823 0.0362 0.0892 -0.0831 -0.0747 0.1039 -0.0266 0.4814 0.0162 -0.0484 -0.0033 0.0761 -0.0312 0.0213 -0.0188 0.0121 -0.0537 0.0473 0.0583 0.0292 0.0655 0.0111 0.1129 0.0659 -0.0759 0.0795 -0.0425 -0.0335 -0.1145 0.01 0.0197 -0.1981 -0.0385 0.0319 0.0612 0.0273 -0.0384 0.035 -0.0085 -0.2808 -0.0483 0.1094 0.0

**La suite:**
1. Calcul du nouveau score grâce aux embeddings de mots des titres pour les 2900 couples
2. On tire au hasard 100 couples à partir des 2900 couples: 10 couples pour chaque décile
3. Annotation de ces 100 couples grâce à l'application Shiny
4. Avec le programme .R : nouvelle courbe des % de match bien prédits pour chaque décile
5. Avec le programme .R : nouvelle courbe de ROC basée sur le nouveau score

Calcul du cosine similarity pour ce couple de produits

In [0]:
def cosine_similarity(title_one,title_two):
    # calcul de l'embedding du titre 1
    sum_of_embeddings_one = 0
    for word in title_one.split(' '):
        #print (word)
        sum_of_embeddings_one += word_to_vec_map_words_in_titles_bio[word]
    
    # On calcule en fait la moyenne des embeddings car on peut avoir un nombre de mots différent
    # entre les deux titres
    sum_of_embeddings_one = sum_of_embeddings_one / len(title_one.split(' '))
    
    #print (np.expand_dims(sum_of_embeddings_one, axis = 1).shape)
        
    # calcul de l'embedding du titre 2
    sum_of_embeddings_two = 0
    for word in title_two.split(' '):
        #print (word)
        sum_of_embeddings_two += word_to_vec_map_words_in_titles_bio[word]
    #print (np.expand_dims(sum_of_embeddings_one, axis = 1).shape)
    
    # ajouter la moyenne
    sum_of_embeddings_two = sum_of_embeddings_two / len(title_two.split(' '))
    
    u = np.expand_dims(sum_of_embeddings_one, axis = 1) 
    v = np.expand_dims(sum_of_embeddings_two, axis = 1)
    resultat = np.sum(u*v) / (np.linalg.norm(u)*np.linalg.norm(v))
    #resultat = sklearn.metrics.pairwise.cosine_similarity(u,v)          
    return resultat

In [0]:
title_1 = "Baguette Classique Sans Gluten 360g Bio"
title_2 = "Cake Citron sans gluten 200gr"
cosine_similarity(title_1,title_2)

0.7174120672064965

In [0]:
title_1 = "Bio flan Citron 2x1/4l"
title_2 = "Bio Bioflan Citron sans sucre 7g"
cosine_similarity(title_1,title_2)

0.6732846897099303

In [0]:
title_1 = "Tamari Strong 145ml Bio"
title_2 = "Sauce Soja Tamari"
cosine_similarity(title_1,title_2)

0.6246775866959284

In [0]:
1. On charge les 2900 couples dans un dataframe

In [0]:
# df_all_couples = pd.read_excel("data/all_results_BIO_with_exhaust_and_titles_equal_20190529_124456_all.xlsx") 
df_all_couples = pd.read_pickle("data/all_results_final_all_with_filter.pkl")

In [0]:
df_all_couples.shape

(2950, 17)

In [0]:
df_all_couples.shape

(2964, 17)

2. Calcul du nouveau score grâce aux embeddings de mots des titres

In [0]:
df_all_couples['xtitle_one_without_xa0'] = df_all_couples['xtitle_one'].map(lambda x:re.sub('\xa0',' ',x))
df_all_couples['xtitle_two_without_xa0'] = df_all_couples['xtitle_two'].map(lambda x:re.sub('\xa0',' ',x))

In [0]:
def compute_cosine_similarity(df):
    resultat = cosine_similarity(df["xtitle_one_without_xa0"],df["xtitle_two_without_xa0"])
    return resultat

In [0]:
df_all_couples["cosine_similarity_of_titles"] = df_all_couples.apply(compute_cosine_similarity, axis = 1)

In [0]:
df_all_couples.sample(n=1)

,id_entities,couples,slug_one,slug_two,xtitle_one,xtitle_two,distributeur_one,distributeur_two,brand_one,brand_two,breadcrumb_one,breadcrumb_two,price_one,price_two,URL_one,URL_two,confidence_score,xtitle_one_without_xa0,xtitle_two_without_xa0,cosine_similarity_of_titles
181,"(manuka, citron, gingembre, 20, sachets)","[[tisane-citr-ging-manuka-20inf, Tisane Citron Gingembre Manuka - 20 Sachets Bio, ntr, Pukka, Infusions ayurvédiques, 3.39, tisane citron gingembre manuka 20 sachets bio, 0.7142857142857143], [pukka-tisane-citron-gingembre-et-miel-de-manuka-bio-20-sachets-p80056, Tisane Citron, Gingembre et Miel de Manuka bio - 20 sachets, gwz, Pukka, Tisanes ayurvédiques, 3.51, tisane citron gingembre miel manuka 20 sachets, 0.7142857142857143]]",tisane-citr-ging-manuka-20inf,pukka-tisane-citron-gingembre-et-miel-de-manuka-bio-20-sachets-p80056,Tisane Citron Gingembre Manuka - 20 Sachets Bio,"Tisane Citron, Gingembre et Miel de Manuka bio - 20 sachets",ntr,gwz,Pukka,Pukka,Infusions ayurvédiques,Tisanes ayurvédiques,3.39,3.51,https://naturalia.fr/tisane-citr-ging-manuka-20inf,https://www.greenweez.com/pukka-tisane-citron-gingembre-et-miel-de-manuka-bio-20-sachets-p80056,0.714286,Tisane Citron Gingembre Manuka - 20 Sachets Bio,"Tisane Citron, Gingembre et Miel de Manuka bio - 20 sachets",0.901005


In [0]:
df_all_couples[["xtitle_one","xtitle_two","cosine_similarity_of_titles"]].head(n=20)

,xtitle_one,xtitle_two,cosine_similarity_of_titles
0,Graines à Germer Bio RADIS NOIR 150g,Radis Noir Graines à Germer,0.820172
3,Gluten Pur,Gluten Pur,1.000000
0,Chips Lentilles Original 90g,Chips aux Lentilles Original 90g,0.884189
4,Sardines à l'huile d'olive,Sardines à l'Huile d'Olive,0.957535
1,Couscous Demi-complet 500g,Couscous Demi-complet 500g,1.000000
2,Bouillon cube Légumes aux Herbes,Bouillon de Légumes aux Herbes Cube,0.856759
1,Crème de Riz pour bébé,Crème de riz pour bébé,0.927083
2,Sésame Noir - 250g,Sésame Noir - 250g,1.000000
5,Café Grains BIO - Sélection 100% Arabica 1kg,Café Grains 100% Arabica,0.708431
16,Soupe Potimarron & courge Butternut 25cl,Soupe Potimarron & Courge Butternut,0.933894


Calcul de la nouvelle colonne strate pour chaque décile de "cosine_similarity_of_titles"

In [0]:
x_values = list(range(0,100,10))
perc_values = [df_all_couples[['cosine_similarity_of_titles']].quantile(x/100)[0] for x in x_values]

In [0]:
def strate_percentiles(x):
    
    if x > 0 and x <= perc_values[0]:
        resultat = '1'
#         print (resultat)
    elif x > perc_values[0] and x <= perc_values[1]:
        resultat = '1'
#         print (resultat)
    elif x > perc_values[1] and x <= perc_values[2]:
        resultat = '2'
#         print (resultat)
    elif x > perc_values[2] and x <= perc_values[3]:
        resultat = '3'
#         print (resultat)
    elif x > perc_values[3] and x <= perc_values[4]:
        resultat = '4' 
#         print (resultat)
    elif x > perc_values[4] and x <= perc_values[5]:
        resultat = '5'
#         print (resultat)
    elif x > perc_values[5] and x <= perc_values[6]:
        resultat = '6'
#         print (resultat)
    elif x > perc_values[6] and x <= perc_values[7]:
        resultat = '7'
#         print (resultat)
    elif x > perc_values[7] and x <= perc_values[8]:
        resultat = '8'
#         print (resultat)
    elif x > perc_values[8] and x <= perc_values[9]:
        resultat = '9'
#         print (resultat)
#     elif x > perc_values[9] and x <= 1:
    else :
        resultat = '10'
#         print (resultat)

    return resultat

df_all_couples['strate_var_cosine_similarity'] = df_all_couples['cosine_similarity_of_titles'].map(strate_percentiles)

In [0]:
df_all_couples[df_all_couples["strate_var_cosine_similarity"] == '1']

,id_entities,couples,slug_one,slug_two,xtitle_one,xtitle_two,distributeur_one,distributeur_two,brand_one,brand_two,breadcrumb_one,breadcrumb_two,price_one,price_two,URL_one,URL_two,confidence_score,xtitle_one_without_xa0,xtitle_two_without_xa0,cosine_similarity_of_titles,strate_var_cosine_similarity
75,"(puree, complete, d'amande)","[[4503-puree-d-amandes-completes-bio-350-g-3390390000115?PageSpeed=noscript, Purée d'Amande Complète, wbe, Jean Hervé, Pâte à tartiner bio, 8.89, puree d'amande complete, 1.0], [jean-herve-puree-d-amande-complete-5kg-p73162, Puree d'amande complète 5kg, gwz, Jean Hervé, Purées d'amandes, 114.03, puree d'amande complete 5kg, 0.75]]",jean-herve-puree-d-amande-complete-5kg-p73162,4503-puree-d-amandes-completes-bio-350-g-3390390000115?PageSpeed=noscript,Puree d'amande complète 5kg,Purée d'Amande Complète,gwz,wbe,Jean Hervé,Jean Hervé,Purées d'amandes,Pâte à tartiner bio,114.03,8.89,https://www.greenweez.com/jean-herve-puree-d-amande-complete-5kg-p73162,https://www.webecologie.com/pate-a-tartiner-bio/4503-puree-d-amandes-completes-bio-350-g-3390390000115.html?PageSpeed=noscript,0.875000,Puree d'amande complète 5kg,Purée d'Amande Complète,0.436755,1
76,"(puree, complete, d'amande)","[[4502-puree-d-amandes-completes-bio-700-g-3390390000122?PageSpeed=noscript, Purée d'Amande Complète, wbe, Jean Hervé, Pâte à tartiner bio, 17.43, puree d'amande complete, 1.0], [jean-herve-puree-d-amande-complete-5kg-p73162, Puree d'amande complète 5kg, gwz, Jean Hervé, Purées d'amandes, 114.03, puree d'amande complete 5kg, 0.75]]",jean-herve-puree-d-amande-complete-5kg-p73162,4502-puree-d-amandes-completes-bio-700-g-3390390000122?PageSpeed=noscript,Puree d'amande complète 5kg,Purée d'Amande Complète,gwz,wbe,Jean Hervé,Jean Hervé,Purées d'amandes,Pâte à tartiner bio,114.03,17.43,https://www.greenweez.com/jean-herve-puree-d-amande-complete-5kg-p73162,https://www.webecologie.com/pate-a-tartiner-bio/4502-puree-d-amandes-completes-bio-700-g-3390390000122.html?PageSpeed=noscript,0.875000,Puree d'amande complète 5kg,Purée d'Amande Complète,0.436755,1
121,"(chocolat noir, tartelettes)","[[bonneterre-tartelettes-chocolat-noir-bio-125g-p60368, Tartelettes chocolat noir bio 125g, gwz, Bonneterre, Autres biscuits, 3.19, tartelettes chocolat noir 125g, 0.75], [5955-tartelettes-au-chocolat-noir-3396413710092, Tartelettes au Chocolat Noir, wbe, Bonneterre, Biscuit bio, 3.30, tartelettes chocolat noir, 1.0]]",bonneterre-tartelettes-chocolat-noir-bio-125g-p60368,5955-tartelettes-au-chocolat-noir-3396413710092,Tartelettes chocolat noir bio 125g,Tartelettes au Chocolat Noir,gwz,wbe,Bonneterre,Bonneterre,Autres biscuits,Biscuit bio,3.19,3.30,https://www.greenweez.com/bonneterre-tartelettes-chocolat-noir-bio-125g-p60368,https://www.webecologie.com/biscuit-bio/5955-tartelettes-au-chocolat-noir-3396413710092.html,0.875000,Tartelettes chocolat noir bio 125g,Tartelettes au Chocolat Noir,0.438248,1
120,"(citron, safran, tortils)","[[2977-tortils-citron-safran-bio-3380390173506, Tortils Citron & Safran, wbe, Priméal, Pâtes bio, 3.15, tortils citron & safran, 0.75], [primeal-tortils-au-citron-et-au-safran-p5676, Tortils au Citron et au Safran, gwz, Priméal, Pâtes Parfumées, 2.82, tortils citron safran, 1.0]]",primeal-tortils-au-citron-et-au-safran-p5676,2977-tortils-citron-safran-bio-3380390173506,Tortils au Citron et au Safran,Tortils Citron & Safran,gwz,wbe,Priméal,Priméal,Pâtes Parfumées,Pâtes bio,2.82,3.15,https://www.greenweez.com/primeal-tortils-au-citron-et-au-safran-p5676,https://www.webecologie.com/pates-bio/2977-tortils-citron-safran-bio-3380390173506.html,0.875000,Tortils au Citron et au Safran,Tortils Citron & Safran,0.412378,1
161,sesame decortique,"[[markal-sesame-decortique-250g-p77434, Sésame décortiqué - 250g, gwz, Markal, Graines de Sésame, 2.04, sesame decortique 250g, 0.6666666666666666], [3142-sesame-decortique-bio-3329488521256, Sésame Décortiqué, wbe, Markal, Graines bio & Graines à germer, 2.05, sesame decortique, 1.0]]",markal

In [0]:
def return_sampled_df_by_strate(df_input, value_var): 
    df = df_input[df_input['strate_var_cosine_similarity'] == value_var].sample(n=10)
    return df

In [0]:
all_sampled_100_couples_cos_sim = pd.concat([return_sampled_df_by_strate(df_all_couples,'1'),
                        return_sampled_df_by_strate(df_all_couples,'2'),
                        return_sampled_df_by_strate(df_all_couples,'3'),
                        return_sampled_df_by_strate(df_all_couples,'4'),
                        return_sampled_df_by_strate(df_all_couples,'5'),
                        return_sampled_df_by_strate(df_all_couples,'6'),
                        return_sampled_df_by_strate(df_all_couples,'7'),
                        return_sampled_df_by_strate(df_all_couples,'8'),
                        return_sampled_df_by_strate(df_all_couples,'9'),
                        return_sampled_df_by_strate(df_all_couples,'10')])

In [0]:
all_sampled_100_couples_cos_sim

,id_entities,couples,slug_one,slug_two,xtitle_one,xtitle_two,distributeur_one,distributeur_two,brand_one,brand_two,breadcrumb_one,breadcrumb_two,price_one,price_two,URL_one,URL_two,confidence_score,xtitle_one_without_xa0,xtitle_two_without_xa0,cosine_similarity_of_titles,strate_var_cosine_similarity
258,"(coquillettes, demi-completes)","[[2975-coquillettes-demi-completes-bio-3380390181402, Coquillettes Demi-Complètes, wbe, Priméal, Pâtes bio, 1.75, coquillettes demi-completes, 1.0], [primeal-coquillettes-demi-completes-5kg-p108288, Coquillettes demi-complètes 5kg, gwz, Priméal, Pâtes Demi-Complètes, 15.20, coquillettes demi-completes 5kg, 0.6666666666666666]]",primeal-coquillettes-demi-completes-5kg-p108288,2975-coquillettes-demi-completes-bio-3380390181402,Coquillettes demi-complètes 5kg,Coquillettes Demi-Complètes,gwz,wbe,Priméal,Priméal,Pâtes Demi-Complètes,Pâtes bio,15.20,1.75,https://www.greenweez.com/primeal-coquillettes-demi-completes-5kg-p108288,https://www.webecologie.com/pates-bio/2975-coquillettes-demi-completes-bio-3380390181402.html,0.833333,Coquillettes demi-complètes 5kg,Coquillettes Demi-Complètes,0.323199,1
10,"(cacao, semoule)","[[9157-gourdes-pour-bebe-creme-semoule-cacao-4x-85g-3288131540801, Gourdes Crème Semoule Cacao, wbe, Babybio, Gourde & Doypack, 4.35, gourdes creme semoule cacao, 0.5], [desserts-bebe-des-8-mois-semoule-cacao-babybio-3288131540801, Desserts bébé dès 8 mois, semoule cacao BABYBIO, crf, Babybio, Desserts lactés, 3.11, desserts bebe 8 mois semoule cacao babybio, 0.2857142857142857]]",desserts-bebe-des-8-mois-semoule-cacao-babybio-3288131540801,9157-gourdes-pour-bebe-creme-semoule-cacao-4x-85g-3288131540801,"Desserts bébé dès 8 mois, semoule cacao BABYBIO",Gourdes Crème Semoule Cacao,crf,wbe,Babybio,Babybio,Desserts lactés,Gourde & Doypack,3.11,4.35,https://www.carrefour.fr/p/desserts-bebe-des-8-mois-semoule-cacao-babybio-3288131540801,https://www.webecologie.com/gourde-doypack/9157-gourdes-pour-bebe-creme-semoule-cacao-4x-85g-3288131540801.html,0.392857,"Desserts bébé dès 8 mois, semoule cacao BABYBIO",Gourdes Crème Semoule Cacao,0.479838,1
41,aubergines,"[[7807-assiette-nuit-aubergines-de-provence-et-macaroni-3288131530758?PageSpeed=noscript, Assiette Aubergines de Provence façon Parmigiana & Macaroni, wbe, Babybio, Assiette & Bol, 2.30, assiette aubergines provence parmigiana & macaroni, 0.16666666666666666], [plat-bebe-des-8-mois-aubergines-babybio-3288131520513, Plat bébé dès 8 mois, aubergines BABYBIO, crf, Babybio, Plats complets, 2.56, plat bebe 8 mois aubergines babybio, 0.16666666666666666]]",plat-bebe-des-8-mois-aubergines-babybio-3288131520513,7807-assiette-nuit-aubergines-de-provence-et-macaroni-3288131530758?PageSpeed=noscript,"Plat bébé dès 8 mois, aubergines BABYBIO",Assiette Aubergines de Provence façon Parmigiana & Macaroni,crf,wbe,Babybio,Babybio,Plats complets,Assiette & Bol,2.56,2.30,https://www.carrefour.fr/p/plat-bebe-des-8-mois-aubergines-babybio-3288131520513,https://www.webecologie.com/assiette-bol/7807-assiette-nuit-aubergines-de-provence-et-macaroni-3288131530758.html?PageSpeed=noscript,0.166667,"Plat bébé dès 8 mois, aubergines BABYBIO",Assiette Aubergines de Provence façon Parmigiana & Macaroni,0.442586,1
1180,melange,"[[3924-melange-pour-poisson-bio-3417960022800, Mélange pour Poisson, wbe, Cook, Épice & Condiment bio, 2.20, melange poisson, 0.5], [cook-melange-grillades-bio-70-g-p94045, Mélange Grillades bio - 70 g, gwz, Cook, Mélanges d'épices, 4.21, melange grillades 70 g, 0.25]]",cook-melange-grillades-bio-70-g-p94045,3924-melange-pour-poisson-bio-3417960022800,Mélange Grillades bio - 70 g,Mélange pour Poisson,gwz,wbe,Cook,Cook,Mélanges d'épices,Épice & Condiment bio,4.21,2.20,https://www.greenweez.com/cook-melange-grillades-bio-70-g-p94045,https://www.webecologie.com/epice-condiment-bio/3924-melange-pour-poisson-bio-3417960022800.html,0.375000,Mélange Grillades bio - 70 g,Mélange pour Poisson,0.319047,1
454,"(flocons, d'orge)","[[3825-flocons-d-orge-bio

In [0]:
output_file_xlsx = "all_sampled_100_couples_cos_sim.xlsx"
all_sampled_100_couples_cos_sim.to_excel(output_file_xlsx, index=False)

**On peut annoter ces 100 nouveaux couples en vue de voir si ce modèle est plus performant ?**

**Conclusion au sujet des différents modèles réalisables:**<br>
avec mesure de la performance du modèle pour 100 couples

In [0]:
On va avoir alors:<br>
- modèle n°1 fait cf Github
- modèle n°2 avec similarité calculée grâce aux word embeddings (FastText)
- modèle n°3 avec similarité calculée grâce aux entités embeddings (Word2vec + transfer learning)
- modèle n°4 avec similarité calculée grâce aux entités embeddings (Bert + transfer learning)
- modèle n°5 avec similarité calculée grâce aux encodings des images (architecture siamese network (à vérifier) + transfer learning)